In [28]:
import open3d as o3d
# open two_spheres.stl file in open3d and visualize
mesh = o3d.io.read_triangle_mesh("data/n_spheres.stl")
mesh.compute_vertex_normals()  # Compute normals if not present
o3d.visualization.draw_geometries([mesh])

In [35]:
# load tissue.brep into the occ kernel in gmsh and plot it 
import gmsh
import numpy as np

def get_bbox(dim, tag):
    xmin, ymin, zmin, xmax, ymax, zmax = gmsh.model.getBoundingBox(dim, tag)
    bbox_min = np.array([xmin, ymin, zmin])
    bbox_max = np.array([xmax, ymax, zmax])
    bbox_center = (bbox_min + bbox_max) / 2
    bbox_size = bbox_max - bbox_min
    return bbox_center, bbox_size

gmsh.initialize()
gmsh.model.add("BboxCut")
shape = gmsh.model.occ.importShapes("data/sphere.brep")
gmsh.model.occ.synchronize()
# translate so bbox center is at (0, 0, L/2)
center, size = get_bbox(shape[0][0], shape[0][1])
gmsh.model.occ.translate(shape, -center[0], -center[1], -(center[2] - size[2]/2))
center, size = get_bbox(shape[0][0], shape[0][1])
# scale so that the height is 1
scale = 1.0 / size[2]  # size[2] is the z-extent
gmsh.model.occ.dilate(shape, 0, 0, 0, scale, scale, scale)
gmsh.model.occ.synchronize()


# draw a box matching the bbox of the shape
center, size = get_bbox(shape[0][0], shape[0][1])
box = gmsh.model.occ.addBox(-size[0]/2, -size[1]/2, 0, size[0], size[1], size[2])
gmsh.model.occ.synchronize()
# cut the box with the shape
volumes = gmsh.model.getEntities(dim=3)
print(volumes)
# cut all volumes from the box
for vol in volumes:
    if vol[1] != box:
        gmsh.model.occ.cut([(3, box)], [vol])
gmsh.model.occ.synchronize()
volumes = gmsh.model.getEntities(dim=3)
print(volumes)
# generate mesh
gmsh.model.mesh.generate(3)
gmsh.fltk.run()
gmsh.finalize()


[(3, 1), (3, 2)]
[(3, 2)]                                                                                                                                   
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (TrimmedCurve)
Info    : [ 10%] Meshing curve 2 (TrimmedCurve)
Info    : [ 10%] Meshing curve 3 (TrimmedCurve)
Info    : [ 10%] Meshing curve 4 (TrimmedCurve)
Info    : [ 10%] Meshing curve 5 (TrimmedCurve)
Info    : [ 10%] Meshing curve 6 (TrimmedCurve)
Info    : [ 10%] Meshing curve 7 (TrimmedCurve)
Info    : [ 10%] Meshing curve 8 (TrimmedCurve)
Info    : [ 10%] Meshing curve 9 (TrimmedCurve)
Info    : [ 10%] Meshing curve 10 (TrimmedCurve)
Info    : [ 10%] Meshing curve 11 (TrimmedCurve)
Info    : [ 10%] Meshing curve 12 (TrimmedCurve)
Info    : [ 10%] Meshing curve 13 (TrimmedCurve)
Info    : [ 10%] Meshing curve 14 (TrimmedCurve)
Info    : [ 10%] Meshing curve 15 (TrimmedCurve)
Info    : [ 10%] Meshing curve 16 (TrimmedCurve)
Info    : [ 10%] Meshing curve 17 (TrimmedCu

X_ChangeProperty: BadValue (integer parameter out of range for operation) 0x0


-------------------------------------------------------
Version       : 4.14.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20250902
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.11
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.8.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
Info    : Meshing 1D...
Info    : [  0%] Me

Info    : 1127 edge swaps, 227 node relocations (volume = 0.571206): worst = 0.000354642 / average = 0.47893 (Wall 0.0617517s, CPU 0.062917s)
Info    : 0.00 < quality < 0.10 :       287 elements
Info    : 0.10 < quality < 0.20 :       550 elements
Info    : 0.20 < quality < 0.30 :      1030 elements
Info    : 0.30 < quality < 0.40 :      1678 elements
Info    : 0.40 < quality < 0.50 :      1741 elements
Info    : 0.50 < quality < 0.60 :      1285 elements
Info    : 0.60 < quality < 0.70 :      1053 elements
Info    : 0.70 < quality < 0.80 :       737 elements
Info    : 0.80 < quality < 0.90 :       608 elements
Info    : 0.90 < quality < 1.00 :       247 elements
Info    : Done optimizing mesh (Wall 0.0679936s, CPU 0.069563s)
Info    : 2982 nodes 18180 elements
